In [18]:
import os
import pandas as pd
import mindspore
import mindcv
import mindspore.nn as nn
import mindspore.dataset.transforms as transforms
import mindspore.dataset.vision as vision
from d2l import mindspore as d2l

In [2]:
#@save
d2l.DATA_HUB['dog_tiny'] = (d2l.DATA_URL + 'kaggle_dog_tiny.zip',
                            '0cb91d09b814ecdc07b50f31f8dcad3e81d6a86d')

# 如果使用Kaggle比赛的完整数据集，请将下面的变量更改为False
demo = True
if demo:
    data_dir = d2l.download_extract('dog_tiny')
else:
    data_dir = os.path.join('..', 'data', 'dog-breed-identification')

In [4]:
def reorg_dog_data(data_dir, valid_ratio):
    labels = d2l.read_csv_labels(os.path.join(data_dir, 'labels.csv'))
    d2l.reorg_train_valid(data_dir, labels, valid_ratio)
    d2l.reorg_test(data_dir)


batch_size = 32 if demo else 128
valid_ratio = 0.1
reorg_dog_data(data_dir, valid_ratio)

In [5]:
transform_train = transforms.Compose([
    # 随机裁剪图像，所得图像为原始面积的0.08～1之间，高宽比在3/4和4/3之间。
    # 然后，缩放图像以创建224x224的新图像
    vision.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3.0/4.0, 4.0/3.0)),
    vision.RandomHorizontalFlip(),
    # 随机更改亮度，对比度和饱和度
    vision.RandomColorAdjust(brightness=0.4,
                                       contrast=0.4,
                                       saturation=0.4),
     # 标准化图像的每个通道
    vision.Normalize(mean=[0.485 * 255, 0.456 * 255, 0.406 * 255], 
                     std=[0.229 * 255, 0.224 * 255, 0.225 * 255]),
    vision.HWC2CHW()])

In [6]:
transform_test = transforms.Compose([
    vision.Resize(256),
    # 从图像中心裁切224x224大小的图片
    vision.CenterCrop(224),
    vision.Normalize([0.485 * 255, 0.456 * 255, 0.406 * 255],
                     [0.229 * 255, 0.224 * 255, 0.225 * 255]),
    vision.HWC2CHW()])

In [7]:
train_ds, train_valid_ds = [mindspore.dataset.ImageFolderDataset(
    os.path.join(data_dir, 'train_valid_test', folder), shuffle=True, decode=True) 
                            for folder in ['train', 'train_valid']]
train_ds = train_ds.map(transform_train, 'image')
train_valid_ds = train_valid_ds.map(transform_train, 'image')


valid_ds, test_ds = [mindspore.dataset.ImageFolderDataset(
    os.path.join(data_dir, 'train_valid_test', folder), shuffle=False, decode=True) 
                     for folder in ['valid', 'test']]
valid_ds = valid_ds.map(transform_test, 'image')
test_ds = test_ds.map(transform_test, 'image')

In [8]:
train_iter, train_valid_iter = [dataset.batch(batch_size=batch_size, drop_remainder=True)
                                for dataset in (train_ds, train_valid_ds)]

valid_iter = valid_ds.batch(batch_size=batch_size, drop_remainder=True)

test_iter = test_ds.batch(batch_size=batch_size, drop_remainder=False)

In [9]:
import mindspore.common.initializer as initializer
def get_net(devices):
    finetune_net = nn.SequentialCell()
    finetune_net.feature = mindcv.create_model('resnet34', pretrained=True)
    #finetune_net.append(feature)
    # 定义一个新的输出网络，共有120个输出类别
    output_new = nn.SequentialCell([nn.Dense(1000, 256),
                  nn.ReLU(),
                  nn.Dense(256, 120)])
    for name, cell in output_new.cells_and_names():
        if isinstance(cell, nn.Dense):
            k = 1 / cell.in_channels
            k = k ** 0.5

            cell.weight.set_data(
                initializer.initializer(initializer.Uniform(k), cell.weight.shape, cell.weight.dtype))
            if cell.bias is not None:
                cell.bias.set_data(
                    initializer.initializer(initializer.Uniform(k), cell.bias.shape, cell.bias.dtype))

    finetune_net.append(output_new)
    #finetune_net.append(output_new)
    # 冻结参数
    for param in finetune_net.feature.get_parameters():
        param.requires_grad = False
    return finetune_net

In [10]:
net = get_net(None)

In [11]:
loss = nn.CrossEntropyLoss(reduction='none')

def evaluate_loss(data_iter, net):
    l_sum, n = 0.0, 0
    for features, labels in data_iter:
        outputs = net(features)
        l = loss(outputs, labels)
        l_sum += l.sum()
        n += labels.numel()
    return (l_sum / n)

In [12]:
def train(net, train_iter, valid_iter, num_epochs, lr, wd, lr_period, lr_decay):
    devices = None
    lr_list = d2l.tensor([lr*(lr_decay**(i//lr_period)) 
                          for i in range(num_epochs) 
                          for j in range(train_iter.get_dataset_size())])
    trainer = nn.SGD((param for param in net.get_parameters() if param.requires_grad), 
                     learning_rate=lr_list, momentum=0.9, weight_decay=wd)

    def forward_fn(inputs, targets):
        logits = net(inputs)
        # print(logits.shape, targets.shape)
        l = loss(logits, targets)
        return l, logits
    
    grad_fn = mindspore.value_and_grad(forward_fn, None, trainer.parameters, has_aux=True)
    
    def train_step(inputs, targets):
        (l, logits), grads = grad_fn(inputs, targets)
        trainer(grads)
        return l.sum(), logits
    
    num_batches, timer = train_iter.get_dataset_size(), d2l.Timer()
    legend = ['train loss']
    if valid_iter is not None:
        legend.append('valid loss')
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=legend)

    for epoch in range(num_epochs):
        net.set_train()
        metric = d2l.Accumulator(2)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, logits = train_step(features, labels)
            metric.add(l, labels.shape[0])
            timer.stop()
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (metric[0] / metric[1], None))
        measures = f'train loss {metric[0] / metric[1]:.3f}'
        if valid_iter is not None:
            valid_loss = evaluate_loss(valid_iter, net)
            animator.add(epoch + 1, (None, valid_loss))

    if valid_iter is not None:
        measures += f', valid loss {float(valid_loss):.3f}'
    print(measures + f'\n{metric[1] * num_epochs / timer.sum():.1f}'
          f' examples/sec on {str(devices)}')

In [13]:
num_epochs, lr, wd = 10, 1e-4, 1e-4
lr_period, lr_decay, net = 2, 0.9, get_net(None)
train(net, train_iter, valid_iter, num_epochs, lr, wd, lr_period,
      lr_decay)

train loss 1.468, valid loss 1.511
13.1 examples/sec on None


In [19]:
net, preds = get_net(None), []
train(net, train_valid_iter, None, num_epochs, lr, wd, lr_period,
      lr_decay)

for X, _ in test_iter:
    y_hat = net(X)
    preds.extend(y_hat.argmax(axis=1).numpy().astype('int32'))
sorted_ids = list(range(1, test_ds.get_dataset_size() + 1))
sorted_ids.sort(key=lambda x: str(x))

df = pd.DataFrame({'id': sorted_ids, 'label': preds})
class_indexing = train_ds.get_class_indexing()
classes = sorted(class_indexing.items(), key=lambda x: x[0])
df['label'] = df['label'].apply(lambda x: classes[x][0])
df.to_csv('submission.csv', index=False)

train loss 1.520
14.3 examples/sec on None
